In [1]:
from acclib.nets.models import ResNetLSTM, ResNetLSTMSteps
from acclib.show import *
from acclib.loader import *
from acclib.train_test_split import *
from acclib.nets.training import *
import torch
from acclib.geometry import *
from acclib.movement_detection.threshold_based import *
from acclib.features import *
from scipy.ndimage import gaussian_filter
from torch import nn

from acclib.helpers import get_windows
from acclib.preprocess import get_trend
import random
from acclib.nets.estimator import *
import pickle

In [2]:
%load_ext autoreload
%autoreload 2

## Without step detection (ResNetLSTM)

In [3]:
data = Storage('/archive/shared_files/acc_to_trj/data/rudacop-public/1_rotated_2')
device = torch.device('cuda:0')


In [4]:
# split = train_test_valid_split('/archive/shared_files/acc_to_trj/data/rudacop-public/1_rotated_2')

In [5]:
#with open('split.dat', 'wb') as dump_out:
#    pickle.dump(split, dump_out)

In [7]:
with open('split.dat', 'rb') as dump_in:
    split = pickle.load(dump_in)


In [8]:
test = []
for i in split['test']:
    test.append(i[-39:])

In [9]:
model = ResNetLSTM([9, 64, 64, 128, 256, 64, 2, 0.01])


In [36]:
estimator = TrajectoryEstimator('ResNetLSTM', model, device = device)

In [37]:
criterion = nn.MSELoss()

In [38]:
estimator.train_model(split['train'], split['valid'], data,save_loss=True, n_epochs = 10, criterion=criterion)

Epoch [1/10], Loss (train/test): 0.136131/0.182483, 
Epoch [2/10], Loss (train/test): 0.078713/0.159238, 
Epoch [3/10], Loss (train/test): 0.061640/0.155431, 
Epoch [4/10], Loss (train/test): 0.051069/0.159779, 
Epoch [5/10], Loss (train/test): 0.043203/0.144995, 
Epoch [6/10], Loss (train/test): 0.037081/0.147467, 
Epoch [7/10], Loss (train/test): 0.032194/0.149752, 
Epoch [8/10], Loss (train/test): 0.028373/0.148486, 
Epoch [9/10], Loss (train/test): 0.024507/0.145752, 
Epoch [10/10], Loss (train/test): 0.021739/0.149177, 


In [39]:
estimator.save()
#estimator.load( )

In [41]:
estimator.evaluate(test, data, print_metrics=True, save_folder='ResNetLSTM')

100%|██████████| 62/62 [04:36<00:00,  4.46s/it]

rmse       8.711713
rte_10     7.092999
rte_30     7.417330
rte_60     7.915545
mie        6.908382
gap       14.543363
dtype: float64


In [11]:
data

## With step detection (ResNetLSTM)

In [10]:
from acclib.nets.training import get_x_from_names_steps

In [19]:
d = get_x_from_names_steps(data.experiment_paths[:10], False, 200, 20, True, data, 20)

In [21]:
d[1][:,-2:].shape

torch.Size([1496, 2])

In [23]:
sum(d[1][:,-1])

tensor(217.)

In [24]:
sum(d[1][:,-2])

tensor(215.)

In [11]:
model = ResNetLSTMSteps([9, 64, 64, 128, 256, 64, 2, 0.01])

In [12]:
from acclib.nets.training import loss_function


In [13]:
estimator = TrajectoryEstimator('ff', model, device = device, step=20)

In [14]:
estimator.load('experiments/ResNetLSTMSteps_loss_20_1/')

In [ ]:
estimator.train_model(split['train'], split['valid'], data, save_loss = True, 
                      step=True, n_epochs =10, criterion=loss_function,weight = 2)

Epoch [1/10], Loss (train/test): 0.107255/0.152969, Loss (train_steps/test_steps): 0.001720/0.000033,
Epoch [2/10], Loss (train/test): 0.054164/0.153292, Loss (train_steps/test_steps): 0.000011/0.000002,


In [16]:
estimator.evaluate(test, data, print_metrics=True, save_folder='ResNetLSTMSteps', step = True)

  0%|          | 0/62 [00:00<?, ?it/s]

tensor([0.4659, 0.4659, 0.4660, 0.4659, 0.4659, 0.4659, 0.4659, 0.4659, 0.4659,
        0.4660, 0.4660, 0.4660, 0.4659, 0.4659, 0.4659, 0.4660, 0.4660, 0.4659,
        0.4660, 0.4659, 0.4660, 0.4660, 0.4659, 0.4659, 0.4659, 0.4659, 0.4659,
        0.4660, 0.4659, 0.4659, 0.4660, 0.4659, 0.4660, 0.4660, 0.4659, 0.4660,
        0.4660, 0.4659, 0.4660, 0.4659, 0.4659, 0.4660, 0.4660, 0.4660, 0.4660,
        0.4660, 0.4660, 0.4660, 0.4659, 0.4660, 0.4660, 0.4660, 0.4660, 0.4660,
        0.4660, 0.4660, 0.4659, 0.4660, 0.4660, 0.4659, 0.4659, 0.4660, 0.4660,
        0.4660, 0.4660, 0.4660, 0.4659, 0.4660, 0.4661, 0.4659, 0.4659, 0.4660,
        0.4660, 0.4659, 0.4659, 0.4659, 0.4659, 0.4660, 0.4659, 0.4659, 0.4660,
        0.4660, 0.4660, 0.4659, 0.4659, 0.4660, 0.4659, 0.4659, 0.4660, 0.4660,
        0.4658, 0.4659, 0.4658, 0.4661, 0.4660, 0.4658, 0.4658, 0.4659, 0.4660,
        0.4659, 0.4659, 0.4660, 0.4660, 0.4660, 0.4660, 0.4660, 0.4660, 0.4660,
        0.4659, 0.4659, 0.4659, 0.4659, 

/home/filippova/Projects/env/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/filippova/Projects/env/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/filippova/Projects/env/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1515: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  average, "true nor predicted", 'F-score is', len(true_sum)
  2%|▏         | 1/62 [00:05<05:37,  5.53s/it]

tensor([0.4655, 0.4655, 0.4656,  ..., 0.4660, 0.4660, 0.4658])
tensor([0.5169, 0.5168, 0.5168,  ..., 0.5166, 0.5166, 0.5168])


/home/filippova/Projects/env/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/filippova/Projects/env/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/filippova/Projects/env/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1515: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  average, "true nor predicted", 'F-score is', len(true_sum)
  3%|▎         | 2/62 [00:18<07:42,  7.71s/it]

KeyboardInterrupt: 